In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])

# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

<ipython-input-1-e069bfd8a143>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])


Item-Item Collaborative Filtering with 

In [4]:
# implement your improved model and print out the RMSE
# Your Code Here...

indicator_mat = (train_mat > 0).astype(float)
num_rating_items = np.sum(indicator_mat, axis=0, keepdims=True)
numer = np.matmul(indicator_mat.T, indicator_mat)  # num_item * num_item
denom = num_rating_items.T + num_rating_items - numer  # num_item * num_item
denom[denom == 0] = 1
J = numer / denom  # num_item * num_item

prediction_mat = train_mat.copy()

num_rating_items[num_rating_items == 0] = 1
mu_items = np.sum(train_mat, axis=0, keepdims=True) / num_rating_items  # 1 * num_item
deviation_mat = (train_mat - mu_items) * indicator_mat
for i in range(num_movie):
    similarities = J[i, :]
    similarities[i] = -1
    N_idx = np.argpartition(similarities, -10)[-10:]
    N_sim = similarities[N_idx]
    prediction_mat[:, i] = np.sum(N_sim.reshape((1, -1)) * deviation_mat[:, N_idx], axis=1) / (np.sum(N_sim) + 1e-10)
prediction_mat += mu_items

indicator_mat = (test_mat > 0).astype(float)

test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)
print(np.argsort(prediction_mat[0, :])[-5:])
print(prediction_mat[0, np.argsort(prediction_mat[0, :])[-5:]])

0.9206510883712525
[3577 3609 1805 3500 3629]
[5.         5.         5.         5.         5.05876619]
